# Evaluation of election results for the Norwegian Parliament

### Hans Ekkehard Plesser / NMBU

This notebook determines the number of seats assigned to each party in the Norwegian parliament based on voting data. It does not determine the distribution of adjustment mandates (utjevningsmandater) to individual election districts.

In [1]:
import pandas as pd
import re

from votes_to_mandates_solution import district_mandates

## Load voting results

This code is based on similar code in Lecture 4.

In [2]:
voting_data = pd.read_csv('2021-09-30_party distribution_1_st_2021.csv',
                          sep=';',
                          usecols=['Fylkenavn', 'Partikode', 'Antall stemmer totalt'])
voting_data.rename({'Fylkenavn': 'District',
                    'Partikode': 'Party',
                    'Antall stemmer totalt': 'Votes'},
                    axis=1, inplace=True)

## Load information about seats per district

Since district the file containing this data (from the week 37 exercises) uses spaces to separate values and several district names contain spaces (and hyphens), reading this data is slightly more challenging.

We use a regular expression to split each line into "everything before the number" and the number at the end of each line.

In [3]:
seats = {}
split_re = re.compile(r'([^\d]+)(\d+)')
with open('seats_table.txt') as sf:
    sf.readline()
    sf.readline()
    for line in sf:
        district, n_seats = split_re.match(line).groups()
        seats[district.strip()] = int(n_seats)
seats

{'Akershus': 19,
 'Aust-Agder': 4,
 'Buskerud': 8,
 'Finnmark Finnmárku': 5,
 'Hedmark': 7,
 'Hordaland': 16,
 'Møre og Romsdal': 8,
 'Nord-Trøndelag': 5,
 'Nordland': 9,
 'Oppland': 6,
 'Oslo': 20,
 'Rogaland': 14,
 'Sogn og Fjordane': 4,
 'Sør-Trøndelag': 10,
 'Telemark': 6,
 'Troms Romsa': 6,
 'Vest-Agder': 6,
 'Vestfold': 7,
 'Østfold': 9}

## Determine mandate distribution in each district

- Use code from the exercise solution
- Collect results as dictionary of dictionaries
- Then convert to dataframe
    - Need to fill missing data with 0 (parties with seats in only some districts)
    - Since we had missing values, dataframe is created with `float` type, we force `int` explicitly

In [4]:
dist_mand_dict = {dist: district_mandates(voting_data.loc[voting_data.District==dist], n_seats)
                  for dist, n_seats in seats.items()}

dist_mand = pd.DataFrame.from_records(dist_mand_dict).fillna(0).astype(int).T
dist_mand

,H,A,FRP,SP,V,SV,MDG,RØDT,PF,KRF
Akershus,5,5,2,2,1,1,1,1,0,0
Aust-Agder,1,1,0,1,0,0,0,0,0,0
Buskerud,2,3,1,1,0,0,0,0,0,0
Finnmark Finnmárku,0,2,0,1,0,0,0,0,1,0
Hedmark,1,3,0,2,0,0,0,0,0,0
Hordaland,4,4,2,2,0,1,0,1,0,1
Møre og Romsdal,1,2,2,2,0,0,0,0,0,0
Nord-Trøndelag,0,2,0,2,0,0,0,0,0,0
Nordland,1,3,1,2,0,1,0,0,0,0
Oppland,1,2,0,2,0,0,0,0,0,0


### Total number of district mandates

- Sum over districts
- Results is series, we convert to frame for easier work below

In [5]:
dist_mand_per_party = dist_mand.sum().to_frame(name='Mandates')
dist_mand_per_party.T

,H,A,FRP,SP,V,SV,MDG,RØDT,PF,KRF
Mandates,35,48,17,28,3,8,3,4,1,3


- Check that we got the right number

In [6]:
dist_mand_per_party.Mandates.sum()

150

## Adjustment mandates

- Compute total number of votes per party throughout Norway

In [7]:
votes_per_party = voting_data.groupby('Party').sum()
votes_per_party.sort_values(by='Votes', ascending=False).T

Party,A,H,SP,FRP,SV,RØDT,V,MDG,KRF,DEMN,...,LIBS,FNB,AAN,PIR,NKP,FI,KYST,GENE,RN,GT
Votes,783394,607316,402961,346474,228063,140931,137433,117647,113344,34068,...,4520,3435,2489,2308,301,275,171,112,97,87


### Function for adjustment

- Pick parties qualified for adjustment by having $\geq 4$% of vote
- Remove parties that have been excluded from adjustment explicitly (see below)
- Subtract number of district mandates won by parties not participating in adjustment from total number of seats to be distributed in adjustment
- Distribute remaining seats among qualifying parties according to Sainte Laguë's rule
    - Note: Since `district_mandates()` subtracts one from number of seats, we need to have `+1` here
        - This is a design weakness of `district_mandates()`
- Combine seats for parties included in adjustment with those not participating to get all seats past adjustment

In [8]:
def determine_adjustment_mands(votes_per_party, dist_mand_per_party, seats_per_district, excluded_parties):
    four_percent_of_votes = 4 * votes_per_party.Votes.sum() / 100
    qualified_parties = set(votes_per_party.loc[votes_per_party.Votes > four_percent_of_votes].index)

    adjusted_parties = qualified_parties - excluded_parties
    unadjusted_parties = set(dist_mand_per_party.index) - adjusted_parties

    votes_per_adjusted_party = votes_per_party.loc[adjusted_parties]

    excluded_seats = dist_mand_per_party.loc[unadjusted_parties].Mandates.sum()
    adjusted_seats = sum(seats_per_district.values()) - excluded_seats
    
    mand_per_adj_party = district_mandates(votes_per_adjusted_party.reset_index(), 
                                           adjusted_seats+1)
    
    
    adjusted_mandates = pd.merge(pd.DataFrame.from_dict(mand_per_adj_party, 
                                                        orient='index', columns=['Mandates']).reset_index(),  
                                 dist_mand_per_party.loc[unadjusted_parties].reset_index(), 
                                 how='outer').set_index('index')
    adjusted_mandates.index.name = 'Party'
    
    return adjusted_mandates

### Repeated adjustment

- The adjustment process can assign a party fewer mandates than the number of district mandates it received (negative number of adjustment mandates)
- Then, that party is excluded and the adjustment process repeated
- This continues until no party has a negative number of adjustment mandates
- For the 2021 election, SP is excluded after the first round, A after the second

In [9]:
excluded_from_adjustment = set()

for _ in range(len(votes_per_party)):
    print(f'\nExcluded in this round {excluded_from_adjustment}')
    adjusted_mand = determine_adjustment_mands(votes_per_party, dist_mand_per_party, seats, excluded_from_adjustment)
    adjment_mand = (adjusted_mand - dist_mand_per_party).dropna()
    print(adjment_mand.T)
    if all(adjment_mand.Mandates >= 0):
        break
        
    excluded_from_adjustment |= set(adjment_mand.loc[adjment_mand.Mandates < 0].index)


Excluded in this round set()
          A  FRP  H  KRF  MDG  PF  RØDT  SP  SV  V
Mandates  0    4  2    0    0   0     5  -3   6  5

Excluded in this round {'SP'}
          A  FRP  H  KRF  MDG  PF  RØDT  SP  SV  V
Mandates -1    4  1    0    0   0     4   0   6  5

Excluded in this round {'SP', 'A'}
          A  FRP  H  KRF  MDG  PF  RØDT  SP  SV  V
Mandates  0    4  1    0    0   0     4   0   5  5


## Final election result

- Compare with the official result at https://valgresultat.no/?type=st&year=2021

In [10]:
adjusted_mand.sort_values(by='Mandates', ascending=False).T

Party,A,H,SP,FRP,SV,RØDT,V,KRF,MDG,PF
Mandates,48,36,28,21,13,8,8,3,3,1
